In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

In [16]:
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

### Escolher as funções disponíveis

In [45]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_groq import ChatGroq

# Data model
class RouteQuery(BaseModel):
    """Encaminhe uma consulta de usuário para o datasource mais relevante."""

    datasource: Literal["vectorstore", "websearch", "agendamento"] = Field(
        ...,
        description="Dada uma pergunta do usuário, escolha encaminhá-la para websearch, vectorstore ou agendamento.",
    )

# LLM with function call
llm = ChatGroq(
    temperature=0,
    model="llama3-70b-8192",
)
structured_llm_router = llm.with_structured_output(RouteQuery)

# Prompt 
system = """Você trabalha na Tech4.ai e é um especialista em encaminhar uma pergunta do usuário para um vectorstore, pesquisa na web ou agendamento de reunião.
O vectorstore contém documentos relacionados a perguntas comuns apenas sobre a empresa (Tech4.ai), tais como missão, visão, valores, cultura, programas internos, políticas
de trabalho remoto, horários, etc. Use o vectorstore para perguntas sobre esses tópicos.
Para perguntas sobre as ferramentas Github, Vscode, Jira e discord use a pesquisa na web.
Para perguntas sobre agendamento de reuniões use o agendamento."""

route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

question_router = route_prompt | structured_llm_router

In [46]:
print(question_router.invoke({"question": "Qual a missão da Tech4.ai?"}))
print(question_router.invoke({"question": "Quero saber os valores da empresa"}))
print(question_router.invoke({"question": "Quero saber o que esperar de um dia de trabalho na Tech4.ai"}))
print(question_router.invoke({"question": "gostaria de agendar uma reunião com o time de desenvolvimento"}))
print(question_router.invoke({"question": "Como posso marcar uma reunião com o time de desenvolvimento?"}))
print(question_router.invoke({"question": "Preciso de ajuda com o Github"}))
print(question_router.invoke({"question": "Como posso usar o Vscode?"}))
print(question_router.invoke({"question": "Como posso usar o Jira?"}))

datasource='vectorstore'
datasource='vectorstore'
datasource='vectorstore'
datasource='agendamento'
datasource='agendamento'
datasource='websearch'
datasource='websearch'
datasource='websearch'


### Vector Database

In [39]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_text_splitters import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings


# Load PDF document
pdf_path = "/home/matgaldino/TECHLAB/Base.pdf"
loader = PyPDFLoader(pdf_path)
docs = loader.load()

# Split the document into chunks
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
doc_splits = text_splitter.split_documents(docs)

# Load an open-source embedding model from Hugging Face
embedding_function = SentenceTransformerEmbeddings(model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Add to vector store
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="Empresa",
    embedding=embedding_function,
)
retriever = vectorstore.as_retriever()

In [41]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
 
# Data model
class GradeDocuments(BaseModel):
    """Pontuação binária para verificar a relevância nos documentos utilizados."""

    binary_score: str = Field(description="Documentos são relevantes para a pergunta, 'sim' or 'não'")

# LLM with function call 
structured_llm_grader_docs = llm.with_structured_output(GradeDocuments)

# Prompt 
system = """Você é um avaliador que avalia a relevância de um documento recuperado para uma pergunta do usuário.
 Se o documento contiver palavra(s)-chave ou significado semântico relacionado à pergunta, classifique-o como relevante. 
 Dê uma pontuação binária "sim" ou "não" para indicar se o documento é relevante para a pergunta."""

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader_relevance = grade_prompt | structured_llm_grader_docs

In [42]:
from langchain_core.output_parsers import StrOutputParser

# Prompt
prompt = ChatPromptTemplate.from_template(
    """Você é um assistente para tarefas de resposta a perguntas. Use as seguintes partes do contexto recuperado para responder à pergunta.
      Se você não souber a resposta, basta dizer que não sabe.
Question: {question}
Context: {context}
Answer:"""
)
 
# Chain
rag_chain = prompt | llm | StrOutputParser()

In [43]:
question = "O que a governança faz?"
docs = retriever.invoke(question)
doc_txt = docs[1].page_content
print(retrieval_grader_relevance.invoke({"question": question, "document": doc_txt}))
# Run
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)
question = "Qual a missão da Apple?"
docs = retriever.invoke(question)
doc_txt = docs[1].page_content
print(retrieval_grader_relevance.invoke({"question": question, "document": doc_txt}))
# Run
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)
question = "De qual circulo é o time de IAG?"
docs = retriever.invoke(question)
doc_txt = docs[1].page_content
print(retrieval_grader_relevance.invoke({"question": question, "document": doc_txt}))
# Run
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

binary_score='sim'
A governança faz uma abordagem proativa para a gestão de tecnologias em toda a organização, garantindo que todas as ferramentas corporativas sejam utilizadas de forma eficaz. Eles identificam oportunidades de melhoria nos processos internos, avaliam a satisfação dos funcionários em cada programa institucional e estão sempre prontos para resolver problemas e comunicar sugestões de aprimoramento para as partes interessadas. Além disso, eles trabalham de forma diligente para assegurar a aplicação dos requisitos das certificações, mantendo todos os documentos necessários atualizados e divulgando as responsabilidades internamente. Em resumo, eles são responsáveis por otimizar o ciclo de vida de desenvolvimento de software, garantindo que os processos sejam ágeis, seguros e eficientes.
binary_score='não'
The question is "Qual a missão da Apple?"

However, the provided context does not mention Apple or its mission. The context appears to be related to a company called TECHL

In [ ]:
question = "Qual a missão da Tech4.ai?"
docs = retriever.invoke(question)
doc_txt = docs[1].page_content
print(retrieval_grader_relevance.invoke({"question": question, "document": doc_txt}))